# 1. Introduction

**Hacktiv8 - Final Project**

**Batch RMT-038**

**Team Members**
- Julita G. Ariana, Data Scientist
- Niko A. Hakam, Data Scientist
- Nisrina Tsany S, Data Analyst
- Ryan Dionenatan, Data Engineer
- Verren Monica, Data Analyst

**About The Project**

Notebook ini berisikan inference proyek ExpedAnalysis dari modeling dengan LDA sebelumnya yang sudah di clustering dan dipisahkan topiknya terkait review bagus dan jelek dari 3 kurir ekspedisi mulai dari JNE, JNT, dan SiCepat yang ada di setiap perwakilan kota di 38 provinsi di Indonesia.

# 2. Import Libraries

In [1]:
# import required libraries
import pandas as pd

from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pickle

from nltk.tokenize import word_tokenize

# from custom script in this dir
from prepro_script import text_preprocessing_id

# 3. Model Inference 

Karena sebelumnya terdapat 2 model yaitu untuk jelek_modeling dan bagus_modeling maka inference ini dibagi menjadi dua bagian.

## 3.1. Jelek Modeling

In [2]:
# Load the saved LDA model
jelek_lda_model = LdaModel.load('jelek_lda_model.model')

# Load the dictionary
jelek_dictionary = Dictionary.load('jelek_lda_dictionary.dict')

# Load the corpus (optional, for validation purposes)
with open('jelek_lda_corpus.pkl', 'rb') as f_jelek:
    jelek_corpus = pickle.load(f_jelek)

print("Model, dictionary, and corpus loaded successfully.")

Model, dictionary, and corpus loaded successfully.


In [3]:
# Memetakan ID topik 
jelek_topic_labels = {
    0: "Pelayan Buruk",
    1: "Delay / Lambat",
    2: "Miskomunikasi Kurir"
}

Dictionary yang memetakan ID topik ke nama deskriptif seperti "Pelayan Buruk", "Delay / Lambat", dan "Miskomunikasi Kurir".

### 3.1.1. Single Inference

In [4]:
# New data of Single review
jelek_new_review = "Paket saya terlambat dan layanan buruk sekali."

In [5]:
# Preprocess the review
jelek_processed_review = await text_preprocessing_id(jelek_new_review)

Ulasan diproses menggunakan fungsi text_preprocessing_id untuk menghasilkan teks yang telah dibersihkan.

In [6]:
# Convert to Bag of Words
bow_vector = jelek_dictionary.doc2bow(jelek_processed_review.split())

# Get topic distribution with labels
topics = jelek_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)

# Display topics with labels
print(f"Review : {jelek_new_review}")
print(f"Processed Review: {jelek_processed_review}")
print()
print("Infered Topics with Labels:")
for topic_id, prob in topics:
    print(f"  {jelek_topic_labels[topic_id]}: {prob:.2%}")

Review : Paket saya terlambat dan layanan buruk sekali.
Processed Review: lambat buruk

Inferred Topics with Labels:
  Pelayan Buruk: 35.21%
  Delay / Lambat: 47.88%
  Miskomunikasi Kurir: 16.91%


Single review inference untuk review jelek ini berhasil dilakukan untuk clustering topic nya. Dimana berdasarkan hasil inference ini, ulasan dari data baru ini paling relevan dengan topik Pelayan Buruk, tetapi juga memiliki keterkaitan yang cukup tinggi dengan topik Delay / Lambat. Hal ini memberikan gambaran bahwa masalah layanan pelanggan dan waktu pengiriman menjadi perhatian utama dalam ulasan ini.

### 3.1.2 Batch Inference

Lalu, dilakukan percobaan untuk melakukan batch inference pada beberapa ulasan baru sekaligus menggunakan model Latent Dirichlet Allocation (LDA) yang telah dibuat untuk mengidentifikasi topik utama dari setiap ulasan berdasarkan distribusi probabilitasnya untuk review jelek.

In [7]:
# batch reviews
jelek_new_reviews = [
    "Paket saya terlambat dan layanan buruk sekali.",
    "Opo toh iki cok. Bangsat. Barangku katut ndi cok. Kntl. Bubar ae. Gak guna!",
    "Kurir sangat tidak sopan dang ngawur. Masak iya kirim tv di taruh di luar pagar rumah. Mengecewakan!",
    "JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂😂",
    """Dima kendala sanak.....lamo bana paket ko..
    Jauh bana simpang haru ko....dari pagi tadi lai....baru jam 12 ba anta paket....sampai kini nda juo tibo2 ala jam 2 lewat lo
    Deh mande mak oi.......""",
    """Ndk sasuai jo namo, paket berhari2 di padang ndk dikirim ka ulak karang, sicepat a namoe tu
    Ancak tutuik se lai""",
    "Iko cocok namonyo diganti jadi SI PANTEK, alamat alah jaleh bakecekan bad adress, sudah tu barang di return baliak, eee kapunduang"
]

In [8]:
for review in jelek_new_reviews:
    processed_review = await text_preprocessing_id(review)
    bow_vector = jelek_dictionary.doc2bow(processed_review.split())
    topics = jelek_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)
    print(f"Review: {review}")
    print(f"Processed Review : {processed_review}")
    print("Inferred Topics with Labels:")
    for topic_id, prob in topics:
        print(f"  {jelek_topic_labels[topic_id]}: {prob:.2%}")
    print()

Review: Paket saya terlambat dan layanan buruk sekali.
Processed Review : lambat buruk
Inferred Topics with Labels:
  Pelayan Buruk: 35.21%
  Delay / Lambat: 47.88%
  Miskomunikasi Kurir: 16.91%

Review: Opo toh iki cok. Bangsat. Barangku katut ndi cok. Kntl. Bubar ae. Gak guna!
Processed Review : barang bagus bubar guna
Inferred Topics with Labels:
  Pelayan Buruk: 19.24%
  Delay / Lambat: 57.63%
  Miskomunikasi Kurir: 23.13%

Review: Kurir sangat tidak sopan dang ngawur. Masak iya kirim tv di taruh di luar pagar rumah. Mengecewakan!
Processed Review : sopan dang sembarang masak tv taruh pagar rumah kecewa
Inferred Topics with Labels:
  Pelayan Buruk: 20.61%
  Delay / Lambat: 35.18%
  Miskomunikasi Kurir: 44.20%

Review: JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂😂
Processed Review : otak mikir orang kalah kaya narapidana
Inferred Topics with Labels:
  Pelayan Buruk: 26.10%
  Delay / Lambat: 44.19%
  Miskomunikasi Kurir: 29.71%

Review: Dima kendala sanak.....lamo bana

- Topik Dominan: Sebagian besar ulasan terkait dengan Delay / Lambat, menunjukkan bahwa keterlambatan pengiriman adalah keluhan utama dalam batch ulasan ini.
- Topik Pendukung: Ada keluhan tambahan tentang Pelayan Buruk dan Miskomunikasi Kurir, yang mengindikasikan masalah terkait layanan pelanggan dan komunikasi kurir.

> Proses batch inference ini membantu untuk menganalisis tema utama dalam banyak ulasan sekaligus, memberikan gambaran menyeluruh tentang area yang perlu diperbaiki.

## 3.2. Bagus Modeling

In [9]:
# Load the saved LDA model
bagus_lda_model = LdaModel.load('bagus_lda_model.model')

# Load the dictionary
bagus_dictionary = Dictionary.load('bagus_lda_dictionary.dict')

# Load the corpus (optional, for validation purposes)
with open('bagus_lda_corpus.pkl', 'rb') as f_bagus:
    bagus_corpus = pickle.load(f_bagus)

print("Model, dictionary, and corpus loaded successfully.")

Model, dictionary, and corpus loaded successfully.


Memberikan label deskriptif pada topik yang dihasilkan oleh model LDA dalam dataset ulasan bagus. Dengan memetakan ID topik (0, 1, 2) ke label seperti "Pelayan Bagus", "Kantor Cabang", dan "Respon Staff", proses ini memudahkan interpretasi hasil analisis topik.

In [10]:
# Memetakan ID topik
bagus_topic_labels = {
    0: "Pelayan Bagus",
    1: "Kantor Cabang",
    2: "Respon Staff"
}

### 3.2.1. Single Inference

In [11]:
# Single review
bagus_new_review = "Tempat bersih, petugas ramah,lokasi yg strategis."

In [12]:
# Preprocess the review
bagus_processed_review = await text_preprocessing_id(bagus_new_review)

In [13]:
# Convert to Bag of Words
bow_vector = bagus_dictionary.doc2bow(bagus_processed_review.split())

# Get topic distribution with labels
topics = bagus_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)

# Display topics with labels
print(f"Review : {bagus_new_review}")
print(f"Processed Review: {bagus_processed_review}")
print()
print("Inferred Topics with Labels:")
for topic_id, prob in topics:
    print(f"  {bagus_topic_labels[topic_id]}: {prob:.2%}")

Review : Tempat bersih, petugas ramah,lokasi yg strategis.
Processed Review: bersih tugas ramah lokasi strategis

Inferred Topics with Labels:
  Pelayan Bagus: 84.50%
  Kantor Cabang: 6.77%
  Respon Staff: 8.73%


Model inference untuk single data dari dataset ulasan bagus ini sudah berhasil. Ulasan ini mayoritas terkait dengan topik Pelayan Bagus, mencerminkan kepuasan pelanggan terhadap aspek pelayanan, lokasi, dan sikap staf. Hal ini menunjukkan keberhasilan perusahaan dalam memberikan pengalaman positif bagi pelanggan.

### 3.2.2. Batch Inference

In [14]:
# batch reviews
bagus_new_reviews = [
    "pelayanan kantornya sih oke, kalo servis jne sendiri sudah relatif bagus, cuma kalo pas overload ya bisa molor, tapi masih batas wajar. Still the best lah",
    "Pelayanan bagus,. Pengambilan paket juga tidak ribet,. Top pokok e!",
    "Kalau pulang kantor selalu ngelewatin  kantor jne garuda.  Karena letaknya dekat dengan stasiun kemayoran,  di jalan garuda byk macam macam kuliner apalagi pempeknya banyak yg jualan dan endes banget.  Kalau pas pulang malam,  kadang mampir di ayam gepuk depan kantor jne nya.  Meskipun bilang ga pedes tapi itunterasa pedas 😂 …",
    "Lokasi gudangnya strategis, gampang dicari. Petugas securitynya satset.",
    "Banyak2 be urut dada ja sanak brataan nae,, rata2 klw kalsel pake reg yg belanja dilazada yg pake ongkir gratis,,,paling cepat seminggu q,, aku pke yes 3 hari sampai kerumah",
    """JNT Pusat Jambi. Dekat dari Pasar sehingga gak banyak makan waktu untuk ke lokasi. Kalo ada pengiriman barang sering pakai jasa ini. Paket cepat sampai dan amanah. Lokasi parkiran luas, bersih, dan yang terpenting GRATIS !
        Semoga JNT makin sukses ke depannya dan makin menambah lapangan kerja. Aamiin""",
    "Layanan bagus, ruangan sangat nyaman, menyediakan tempat bermain untuk anak. Lokasi mudah ditemukan. Parkiran kendaraan lumayan luas. Overall sangat direkomendasikan...",
    "004207447425 dari tgl 20 Mei barang udah nyampe di gudang,tapi sampai hari ini 23 Mei belum di antar.... tolong dong barang itu sangat di butuhkan",
    "The waiting area is spacious and comfortable. Fast and friendly service"
]

In [15]:
for review in bagus_new_reviews:
    processed_review = await text_preprocessing_id(review)
    bow_vector = bagus_dictionary.doc2bow(processed_review.split())
    topics = bagus_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)
    print(f"Review: {review}")
    print(f"Processed Review : {processed_review}")
    print("Inferred Topics with Labels:")
    for topic_id, prob in topics:
        print(f"  {bagus_topic_labels[topic_id]}: {prob:.2%}")
    print()

Review: pelayanan kantornya sih oke, kalo servis jne sendiri sudah relatif bagus, cuma kalo pas overload ya bisa molor, tapi masih batas wajar. Still the best lah
Processed Review : layan kantor servis relatif bagus pas overload molor batas wajar 
Inferred Topics with Labels:
  Pelayan Bagus: 88.29%
  Kantor Cabang: 5.00%
  Respon Staff: 6.72%

Review: Pelayanan bagus,. Pengambilan paket juga tidak ribet,. Top pokok e!
Processed Review : layan bagus ambil ribet e
Inferred Topics with Labels:
  Pelayan Bagus: 75.45%
  Kantor Cabang: 6.79%
  Respon Staff: 17.76%

Review: Kalau pulang kantor selalu ngelewatin  kantor jne garuda.  Karena letaknya dekat dengan stasiun kemayoran,  di jalan garuda byk macam macam kuliner apalagi pempeknya banyak yg jualan dan endes banget.  Kalau pas pulang malam,  kadang mampir di ayam gepuk depan kantor jne nya.  Meskipun bilang ga pedes tapi itunterasa pedas 😂 …
Processed Review : pulang kantor ngelewatin kantor garuda letak stasiun mayor jalan garuda kuli

Ketika data yang di inference merupakan batch pun berhasil dilakukan dengan baik dimana didapatkan:
- Topik Dominan: Pelayan Bagus menjadi topik utama di hampir semua ulasan, dengan probabilitas di atas 50%, mencerminkan apresiasi pelanggan terhadap kualitas pelayanan perusahaan.
- Topik Pendukung:
    - Respon Staff: Menonjol pada ulasan yang menyebutkan interaksi langsung dengan staf atau keamanan.
    - Kantor Cabang: Mendapat perhatian dalam ulasan yang menyebutkan lokasi atau fasilitas kantor.
- Apresiasi: Sebagian besar ulasan menyoroti pelayanan baik, lokasi strategis, dan kenyamanan fasilitas sebagai aspek utama yang memuaskan pelanggan